In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
import cv2
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import os
from keras import layers
import keras
!pip install optuna

# Path to your dataset
dataset_path_train = "/content/drive/Shareddrives/Internship data sets/Training"
dataset_path_val = "/content/drive/Shareddrives/Internship data sets/Validation"


# Load the dataset
batch_size = 32
img_height = 256  # Change this to your image height
img_width = 256   # Change this to your image width
#training presets

train_ds = image_dataset_from_directory(
    dataset_path_train,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    dataset_path_val,
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00
Found 177 files belonging to 26 classes.
Found 59 files belonging to 10 classes.


Model 1

In [ ]:
import optuna
import cv2
import numpy as np

class GaborFilterTuner:
    def __init__(self, train_ds, val_ds):
        self.train_ds = train_ds
        self.val_ds = val_ds

    def apply_gabor_filter(self, img, kernel_size, sigma, theta, lambda_, gamma, psi):
        # Apply Gabor filter to the image
        if img.ndim == 2:  # Check if the image is already grayscale
            img_gray = img
        else:  # Convert image to grayscale before applying the filter
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        g_kernel = cv2.getGaborKernel(kernel_size, sigma, theta, lambda_, gamma, psi, ktype=cv2.CV_32F)
        filtered_img = cv2.filter2D(img_gray, -1, g_kernel)
        return filtered_img

    def objective(self, trial):
        kernel_size = trial.suggest_categorical('kernel_size', [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)])
        sigma = trial.suggest_float('sigma', 1.0, 10.0)
        theta = trial.suggest_categorical('theta', [0, 15, 30, 45, 60, 75, 90])
        lambda_ = trial.suggest_float('lambda_', 1.0, 10.0)
        gamma = trial.suggest_float('gamma', 0.5, 2.0)
        psi = trial.suggest_categorical('psi', [0, 15, 30, 45, 60, 75, 90])

        # Apply Gabor filter to the training and validation data
        X_train_pairs, y_train_pairs = [], []
        y_train_labels = []
        X_val_pairs, y_val_pairs = [], []
        all_images = []
        all_labels = []

        for batch in train_ds:
            images, labels = batch
            all_images.extend(images)
            all_labels.extend(labels)

        for i in range(len(all_images)):
            img_A_org, label_A = all_images[i], all_labels[i]
            img_A = self.apply_gabor_filter(img_A_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
            #print("A is: " ,all_labels[i])

            num = 0

            # Create pairs with other images
            for j in range(i, len(all_images)):
                img_B_org, label_B = all_images[j], all_labels[j]
                img_B = self.apply_gabor_filter(img_B_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
                #print("B is: " ,all_labels[j])
                if label_A == label_B:
                    new_label = 1  # same class, label is 1
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                else:
                    new_label = 0  # different class, label is 0

                    if num==0 or num==1 or num==2:
                      num+=1
                    elif num==3:
                      num-=3
                      X_train_pairs.append([img_A, img_B])
                      y_train_pairs.append(new_label)


        all_images = []
        all_labels = []

        for batch in val_ds:
            images, labels = batch
            all_images.extend(images)
            all_labels.extend(labels)

        for i in range(len(all_images)):
            img_A_org, label_A = all_images[i], all_labels[i]
            img_A = self.apply_gabor_filter(img_A_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)

            num=0

            # Create pairs with other images ( non-identical pairs )
            for j in range(i, len(all_images)):
                img_B_org, label_B = all_images[j], all_labels[j]
                img_B = self.apply_gabor_filter(img_B_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
                new_label = int(label_A == label_B)
                #print(new_label)
                if label_A == label_B:
                    new_label = 1  # same class, label is 1
                    X_val_pairs.append([img_B, img_A])
                    y_val_pairs.append(new_label)
                    X_val_pairs.append([img_B, img_A])
                    y_val_pairs.append(new_label)
                else:
                    new_label = 0  # different class, label is 0

                    if num==0 or num==1 or num==2:
                      num+=1
                    elif num==3:
                      num-=3
                      X_val_pairs.append([img_A, img_B])
                      y_val_pairs.append(new_label)


        X_train_pairs = np.array(X_train_pairs)
        y_train_pairs = np.array(y_train_pairs)
        X_val_pairs = np.array(X_val_pairs)
        y_val_pairs = np.array(y_val_pairs)

        X_train_pairs_A = X_train_pairs[:, 0, :, :]  # shape: (619, 256, 256)
        X_train_pairs_B = X_train_pairs[:, 1, :, :]  # shape: (619, 256, 256)
        X_val_pairs_A = X_val_pairs[:, 0, :, :]  # shape: (619, 256, 256)
        X_val_pairs_B = X_val_pairs[:, 1, :, :]  # shape: (619, 256, 256)

        # Create the network
        img_height = 256
        img_width = 256

        dim = 128
        chan = 4

        img_A_inp = Input((img_height, img_width, 1), name='img_A_inp')


        img_B_inp = Input((img_height, img_width, 1), name='img_B_inp')


        def get_cnn_block(depth, size):
                    return Sequential([
                        Conv2D(depth, size, 1, padding = 'same'),
                        BatchNormalization(),
                        Activation('relu'),
                        MaxPooling2D(),
                        Dropout(0.2)
        ])

        DEPTH = 32

        cnn1 = Sequential([
                        get_cnn_block(DEPTH, 7),
                        get_cnn_block(DEPTH*2, 5),
                        get_cnn_block(DEPTH*2, 5),
                        get_cnn_block(DEPTH*4, 3),
        ])

        feature_vector_A = cnn1(img_A_inp)

        feature_vector_B = cnn1(img_B_inp)

        cnn = Sequential([
          GlobalAveragePooling2D(),
          Flatten(),
          Dense(64, activation='relu'),
        ])

        concat = Concatenate()([feature_vector_A, feature_vector_B])

        concat_processed = cnn(concat)

        dense = Dense(64, activation='relu')(concat)
        output = Dense(2, activation='softmax')(dense)

        model = Model(inputs=[img_A_inp, img_B_inp], outputs=output)

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit([X_train_pairs_A, X_train_pairs_B], tf.keras.utils.to_categorical(y_train_pairs, num_classes=2), epochs=30, batch_size=8, verbose=0)

        # Evaluate the model
        y_pred = model.predict([X_val_pairs_A, X_val_pairs_B])
        y_pred_class = np.argmax(y_pred, axis=1)
        accuracy = np.mean(y_pred_class == y_val_pairs)

        return -accuracy

    def tune_hyperparameters(self, n_trials=50):
        study = optuna.create_study(direction='minimize', pruner=MedianPruner())
        study.optimize(self.objective, n_trials=n_trials)
        best_trial = study.best_trial
        best_params = best_trial.params
        best_accuracy = -best_trial.value
        return best_params, best_accuracy



In [ ]:
from optuna.pruners import MedianPruner
# Set the verbosity level to INFO
optuna.logging.set_verbosity(optuna.logging.DEBUG)

# Create an instance of the GaborFilterTuner class
tuner = GaborFilterTuner(train_ds, val_ds)

# Tune the hyperparameters
best_params, best_accuracy = tuner.tune_hyperparameters(n_trials=20)

print("Best parameters:", best_params)
print("Best accuracy:", best_accuracy)

[I 2024-09-12 01:42:56,484] A new study created in memory with name: no-name-3224a974-fc3d-4e02-8832-7c4332104001
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 3) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (5, 5) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (7, 7) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices f

25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 96ms/step


[I 2024-09-12 01:55:52,112] Trial 0 finished with value: -0.8831003811944091 and parameters: {'kernel_size': (5, 5), 'sigma': 2.14647231746076, 'theta': 30, 'lambda_': 5.552071305415156, 'gamma': 1.9451188407851672, 'psi': 45}. Best is trial 0 with value: -0.8831003811944091.


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step


[I 2024-09-12 02:08:28,668] Trial 1 finished with value: -0.8775510204081632 and parameters: {'kernel_size': (5, 5), 'sigma': 2.781211256912874, 'theta': 0, 'lambda_': 2.6061641036454293, 'gamma': 0.6661373467311948, 'psi': 15}. Best is trial 0 with value: -0.8831003811944091.


...

[I 2024-09-12 04:40:23,021] Trial 13 finished with value: -0.8724489795918368 and parameters: {'kernel_size': (11, 11), 'sigma': 9.752920815290551, 'theta': 90, 'lambda_': 2.741585042043635, 'gamma': 1.1328210606945204, 'psi': 75}. Best is trial 8 with value: -0.8924050632911392.


25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 122ms/step


[I 2024-09-12 04:53:00,392] Trial 14 finished with value: -0.851145038167939 and parameters: {'kernel_size': (11, 11), 'sigma': 6.3348255566946445, 'theta': 30, 'lambda_': 6.258114136384048, 'gamma': 1.3174963265791437, 'psi': 0}. Best is trial 8 with value: -0.8924050632911392.


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step


[I 2024-09-12 05:05:35,256] Trial 15 finished with value: -0.8360864040660737 and parameters: {'kernel_size': (7, 7), 'sigma': 8.335708876239508, 'theta': 60, 'lambda_': 3.966324206889052, 'gamma': 1.6138636505170825, 'psi': 60}. Best is trial 8 with value: -0.8924050632911392.


Model 2

In [ ]:
import optuna
import cv2
import numpy as np

class GaborFilterTuner:
    def __init__(self, train_ds, val_ds):
        self.train_ds = train_ds
        self.val_ds = val_ds

    def apply_gabor_filter(self, img, kernel_size, sigma, theta, lambda_, gamma, psi):
        # Apply Gabor filter to the image
        if img.ndim == 2:  # Check if the image is already grayscale
            img_gray = img
        else:  # Convert image to grayscale before applying the filter
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        g_kernel = cv2.getGaborKernel(kernel_size, sigma, theta, lambda_, gamma, psi, ktype=cv2.CV_32F)
        filtered_img = cv2.filter2D(img_gray, -1, g_kernel)
        return filtered_img

    def objective(self, trial):
        kernel_size = trial.suggest_categorical('kernel_size', [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)])
        sigma = trial.suggest_float('sigma', 1.0, 10.0)
        theta = trial.suggest_categorical('theta', [0, 15, 30, 45, 60, 75, 90])
        lambda_ = trial.suggest_float('lambda_', 1.0, 10.0)
        gamma = trial.suggest_float('gamma', 0.5, 2.0)
        psi = trial.suggest_categorical('psi', [0, 15, 30, 45, 60, 75, 90])

        # Apply Gabor filter to the training and validation data
        X_train_pairs, y_train_pairs = [], []
        y_train_labels = []
        X_val_pairs, y_val_pairs = [], []
        all_images = []
        all_labels = []

        for batch in train_ds:
            images, labels = batch
            all_images.extend(images)
            all_labels.extend(labels)

        for i in range(len(all_images)):
            img_A_org, label_A = all_images[i], all_labels[i]
            img_A = self.apply_gabor_filter(img_A_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
            #print("A is: " ,all_labels[i])

            num = 0

            # Create pairs with other images
            for j in range(i, len(all_images)):
                img_B_org, label_B = all_images[j], all_labels[j]
                img_B = self.apply_gabor_filter(img_B_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
                #print("B is: " ,all_labels[j])
                if label_A == label_B:
                    new_label = 1  # same class, label is 1
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                else:
                    new_label = 0  # different class, label is 0

                    if num==0 or num==1 or num==2:
                      num+=1
                    elif num==3:
                      num-=3
                      X_train_pairs.append([img_A, img_B])
                      y_train_pairs.append(new_label)


        all_images = []
        all_labels = []

        for batch in val_ds:
            images, labels = batch
            all_images.extend(images)
            all_labels.extend(labels)

        for i in range(len(all_images)):
            img_A_org, label_A = all_images[i], all_labels[i]
            img_A = self.apply_gabor_filter(img_A_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)

            num=0

            # Create pairs with other images ( non-identical pairs )
            for j in range(i, len(all_images)):
                img_B_org, label_B = all_images[j], all_labels[j]
                img_B = self.apply_gabor_filter(img_B_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
                new_label = int(label_A == label_B)
                #print(new_label)
                if label_A == label_B:
                    new_label = 1  # same class, label is 1
                    X_val_pairs.append([img_B, img_A])
                    y_val_pairs.append(new_label)
                    X_val_pairs.append([img_B, img_A])
                    y_val_pairs.append(new_label)
                else:
                    new_label = 0  # different class, label is 0

                    if num==0 or num==1 or num==2:
                      num+=1
                    elif num==3:
                      num-=3
                      X_val_pairs.append([img_A, img_B])
                      y_val_pairs.append(new_label)


        X_train_pairs = np.array(X_train_pairs)
        y_train_pairs = np.array(y_train_pairs)
        X_val_pairs = np.array(X_val_pairs)
        y_val_pairs = np.array(y_val_pairs)

        X_train_pairs_A = X_train_pairs[:, 0, :, :]  # shape: (619, 256, 256)
        X_train_pairs_B = X_train_pairs[:, 1, :, :]  # shape: (619, 256, 256)
        X_val_pairs_A = X_val_pairs[:, 0, :, :]  # shape: (619, 256, 256)
        X_val_pairs_B = X_val_pairs[:, 1, :, :]  # shape: (619, 256, 256)

        # Create the network
        img_height = 256
        img_width = 256
        dim = 128
        chan = 4

        img_A_inp = Input((img_height, img_width, 1), name='img_A_inp')
        img_A_reshaped = Reshape((dim, dim, chan))(img_A_inp)

        img_B_inp = Input((img_height, img_width, 1), name='img_B_inp')
        img_B_reshaped = Reshape((dim, dim, chan))(img_B_inp)

        def get_cnn_block(depth, size):
            return Sequential([
                Conv2D(depth, size, 1, padding='same'),
                BatchNormalization(),
                Activation('relu'),
                MaxPooling2D(),
                Dropout(0.2)
            ])

        DEPTH = 32

        cnn1 = Sequential([
            get_cnn_block(DEPTH, 3),
            get_cnn_block(DEPTH*2, 3),
            # MaxPooling2D(pool_size=(2, 2))
        ])
        cnn2 = Sequential([
            get_cnn_block(DEPTH, 5),
            get_cnn_block(DEPTH*2, 5),
            # MaxPooling2D(pool_size=(2, 2))
        ])
        cnn3 = Sequential([
            get_cnn_block(DEPTH, 7),
            get_cnn_block(DEPTH*2, 7),
            # MaxPooling2D(pool_size=(2, 2))
        ])

        feature_vector_A1 = cnn1(img_A_reshaped)
        feature_vector_A2 = cnn2(img_A_reshaped)
        feature_vector_A3 = cnn3(img_A_reshaped)

        feature_vector_A = Concatenate()([feature_vector_A1, feature_vector_A2, feature_vector_A3])

        feature_vector_B1 = cnn1(img_B_reshaped)
        feature_vector_B2 = cnn2(img_B_reshaped)
        feature_vector_B3 = cnn3(img_B_reshaped)

        feature_vector_B = Concatenate()([feature_vector_B1, feature_vector_B2, feature_vector_B3])

        cnn = Sequential([
            GlobalAveragePooling2D(),
            Flatten(),
            Dense(64, activation='relu'),
        ])

        feature_vector_A = cnn(feature_vector_A)
        feature_vector_B = cnn(feature_vector_B)

        concat = Concatenate()([feature_vector_A, feature_vector_B])

        dense = Dense(64, activation='relu')(concat)
        output = Dense(2, activation='softmax')(dense)

        model = Model(inputs=[img_A_inp, img_B_inp], outputs=output)

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit([X_train_pairs_A, X_train_pairs_B], tf.keras.utils.to_categorical(y_train_pairs, num_classes=2), epochs=30, batch_size=8, verbose=0)

        # Evaluate the model
        y_pred = model.predict([X_val_pairs_A, X_val_pairs_B])
        y_pred_class = np.argmax(y_pred, axis=1)
        accuracy = np.mean(y_pred_class == y_val_pairs)

        return -accuracy

    def tune_hyperparameters(self, n_trials=50):
        study = optuna.create_study(direction='minimize', pruner=MedianPruner())
        study.optimize(self.objective, n_trials=n_trials)
        best_trial = study.best_trial
        best_params = best_trial.params
        best_accuracy = -best_trial.value
        return best_params, best_accuracy



In [ ]:
from optuna.pruners import MedianPruner
# Set the verbosity level to INFO
optuna.logging.set_verbosity(optuna.logging.DEBUG)

# Create an instance of the GaborFilterTuner class
tuner = GaborFilterTuner(train_ds, val_ds)

# Tune the hyperparameters
best_params, best_accuracy = tuner.tune_hyperparameters(n_trials=20)

print("Best parameters:", best_params)
print("Best accuracy:", best_accuracy)

[I 2024-09-10 08:40:54,882] A new study created in memory with name: no-name-5bb2e843-92fa-4d20-b856-b4b8403d23d4
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 3) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (5, 5) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (7, 7) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices f

25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step


[I 2024-09-10 08:48:40,069] Trial 0 finished with value: -0.8398983481575604 and parameters: {'kernel_size': (11, 11), 'sigma': 1.884211916071171, 'theta': 75, 'lambda_': 3.191836964278021, 'gamma': 1.2144954916173114, 'psi': 30}. Best is trial 0 with value: -0.8398983481575604.


25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 105ms/step


[I 2024-09-10 08:56:03,488] Trial 1 finished with value: -0.7729591836734694 and parameters: {'kernel_size': (11, 11), 'sigma': 8.089736497323775, 'theta': 90, 'lambda_': 5.957348392845492, 'gamma': 1.5701448561889266, 'psi': 15}. Best is trial 0 with value: -0.8398983481575604.


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step


[I 2024-09-10 11:00:03,233] Trial 18 finished with value: -0.861850443599493 and parameters: {'kernel_size': (9, 9), 'sigma': 4.8177100483737325, 'theta': 0, 'lambda_': 2.5270435363530686, 'gamma': 1.9698910586102878, 'psi': 0}. Best is trial 13 with value: -0.8852040816326531.


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step


[I 2024-09-10 11:07:28,168] Trial 19 finished with value: -0.8486005089058524 and parameters: {'kernel_size': (9, 9), 'sigma': 8.158386333089185, 'theta': 15, 'lambda_': 4.4045715226407784, 'gamma': 1.3746980317007038, 'psi': 75}. Best is trial 13 with value: -0.8852040816326531.


Best parameters: {'kernel_size': (9, 9), 'sigma': 6.160567919031775, 'theta': 90, 'lambda_': 1.1222942692597826, 'gamma': 0.6530346050766683, 'psi': 60}
Best accuracy: 0.8852040816326531


Model 3

In [ ]:
import optuna
import cv2
import numpy as np

class GaborFilterTuner:
    def __init__(self, train_ds, val_ds):
        self.train_ds = train_ds
        self.val_ds = val_ds

    def apply_gabor_filter(self, img, kernel_size, sigma, theta, lambda_, gamma, psi):
        # Apply Gabor filter to the image
        if img.ndim == 2:  # Check if the image is already grayscale
            img_gray = img
        else:  # Convert image to grayscale before applying the filter
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        g_kernel = cv2.getGaborKernel(kernel_size, sigma, theta, lambda_, gamma, psi, ktype=cv2.CV_32F)
        filtered_img = cv2.filter2D(img_gray, -1, g_kernel)
        return filtered_img

    def objective(self, trial):
        kernel_size = trial.suggest_categorical('kernel_size', [(3, 3), (5, 5), (7, 7), (9, 9), (11, 11)])
        sigma = trial.suggest_float('sigma', 1.0, 10.0)
        theta = trial.suggest_categorical('theta', [0, 15, 30, 45, 60, 75, 90])
        lambda_ = trial.suggest_float('lambda_', 1.0, 10.0)
        gamma = trial.suggest_float('gamma', 0.5, 2.0)
        psi = trial.suggest_categorical('psi', [0, 15, 30, 45, 60, 75, 90])

        X_train_pairs, y_train_pairs = [], []
        y_train_labels = []
        X_val_pairs, y_val_pairs = [], []
        all_images = []
        all_labels = []

        for batch in train_ds:
            images, labels = batch
            all_images.extend(images)
            all_labels.extend(labels)

        for i in range(len(all_images)):
            img_A_org, label_A = all_images[i], all_labels[i]
            #img_A = enhance_contrast(img_A_org.numpy())
            img_A = self.apply_gabor_filter(img_A_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
            #print("A is: " ,all_labels[i])

            num = 0

            # Create pairs with other images
            for j in range(i, len(all_images)):
                img_B_org, label_B = all_images[j], all_labels[j]
                img_B = self.apply_gabor_filter(img_B_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)
                #print("B is: " ,all_labels[j])
                if label_A == label_B:
                    new_label = 1  # same class, label is 1
                    X_train_pairs.append([img_B, img_A])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)
                    X_train_pairs.append([img_A, img_B])
                    y_train_pairs.append(new_label)

                else:
                    new_label = 0  # different class, label is 0

                    if num==0 or num==1 or num==2:
                      num+=1
                    elif num==3:
                      num-=3
                      X_train_pairs.append([img_A, img_B])
                      y_train_pairs.append(new_label)


        all_images = []
        all_labels = []

        for batch in val_ds:
            images, labels = batch
            all_images.extend(images)
            all_labels.extend(labels)

        for i in range(len(all_images)):
            img_A_org, label_A = all_images[i], all_labels[i]
            img_A = self.apply_gabor_filter(img_A_org.numpy(), kernel_size, sigma, theta, lambda_, gamma, psi)

            num=0

            # Create pairs with other images ( non-identical pairs )
            for j in range(i, len(all_images)):
                img_B_org, label_B = all_images[j], all_labels[j]
                img_B = self.apply_gabor_filter(img_B_org.numpy(),kernel_size, sigma, theta, lambda_, gamma, psi)
                new_label = int(label_A == label_B)
                #print(new_label)
                if label_A == label_B:
                    new_label = 1  # same class, label is 1
                    X_val_pairs.append([img_B, img_A])
                    y_val_pairs.append(new_label)
                    X_val_pairs.append([img_B, img_A])
                    y_val_pairs.append(new_label)
                else:
                    new_label = 0  # different class, label is 0

                    if num==0 or num==1 or num==2:
                      num+=1
                    elif num==3:
                      num-=3
                      X_val_pairs.append([img_A, img_B])
                      y_val_pairs.append(new_label)

        X_train_pairs = np.array(X_train_pairs)
        y_train_pairs = np.array(y_train_pairs)
        X_val_pairs = np.array(X_val_pairs)
        y_val_pairs = np.array(y_val_pairs)

        X_train_pairs_A = X_train_pairs[:, 0, :, :]  # shape: (619, 256, 256)
        X_train_pairs_B = X_train_pairs[:, 1, :, :]  # shape: (619, 256, 256)
        X_val_pairs_A = X_val_pairs[:, 0, :, :]  # shape: (619, 256, 256)
        X_val_pairs_B = X_val_pairs[:, 1, :, :]  # shape: (619, 256, 256)

        # Create the network
        img_height = 256
        img_width = 256
        dim = 128
        chan = 4

        img_A_inp = Input((img_height, img_width, 1), name='img_A_inp')
        img_A_reshaped = Reshape((dim, dim, chan))(img_A_inp)

        img_B_inp = Input((img_height, img_width, 1), name='img_B_inp')
        img_B_reshaped = Reshape((dim, dim, chan))(img_B_inp)

        def get_cnn_block(depth):
            return Sequential([
                Conv2D(depth, 1, padding='same'),
                BatchNormalization(),
                Activation('relu'),
                MaxPooling2D(),
                Dropout(0.2)
            ])

        DEPTH = 32
        cnn = Sequential([
            #data_augmentation,
            get_cnn_block(DEPTH),
            get_cnn_block(DEPTH*2),
            get_cnn_block(DEPTH*4),
            get_cnn_block(DEPTH*8),
            GlobalAveragePooling2D(),
            Flatten(),
            Dense(64, activation='relu')
        ])

        feature_vector_A = cnn(img_A_inp)
        feature_vector_B = cnn(img_B_inp)

        concat = Concatenate()([feature_vector_A, feature_vector_B])

        dense = Dense(64, activation='relu')(concat)
        output = Dense(2, activation='softmax')(dense)

        model = Model(inputs=[img_A_inp, img_B_inp], outputs=output)

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit([X_train_pairs_A, X_train_pairs_B], tf.keras.utils.to_categorical(y_train_pairs, num_classes=2), epochs=30, batch_size=8, verbose=0)

        # Evaluate the model
        y_pred = model.predict([X_val_pairs_A, X_val_pairs_B])
        y_pred_class = np.argmax(y_pred, axis=1)
        accuracy = np.mean(y_pred_class == y_val_pairs)

        return -accuracy

    def tune_hyperparameters(self, n_trials=50):
        study = optuna.create_study(direction='minimize', pruner=MedianPruner())
        study.optimize(self.objective, n_trials=n_trials)
        best_trial = study.best_trial
        best_params = best_trial.params
        best_accuracy = -best_trial.value
        return best_params, best_accuracy



In [ ]:
from optuna.pruners import MedianPruner
# Set the verbosity level to INFO
optuna.logging.set_verbosity(optuna.logging.DEBUG)

# Create an instance of the GaborFilterTuner class
tuner = GaborFilterTuner(train_ds, val_ds)

# Tune the hyperparameters
best_params, best_accuracy = tuner.tune_hyperparameters(n_trials=20)

print("Best parameters:", best_params)
print("Best accuracy:", best_accuracy)

[I 2024-09-09 16:36:24,126] A new study created in memory with name: no-name-c77e34ee-aef1-4d34-a2dd-230a366fee6d
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (3, 3) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (5, 5) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (7, 7) which is of type tuple.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices f

25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 67ms/step


[I 2024-09-09 16:45:57,165] Trial 0 finished with value: -0.7878017789072427 and parameters: {'kernel_size': (9, 9), 'sigma': 7.604297600887105, 'theta': 30, 'lambda_': 9.161830400067139, 'gamma': 0.786988510101412, 'psi': 15}. Best is trial 0 with value: -0.7878017789072427.


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step


[I 2024-09-09 16:55:13,941] Trial 1 finished with value: -0.8316326530612245 and parameters: {'kernel_size': (7, 7), 'sigma': 5.112836726495661, 'theta': 90, 'lambda_': 5.046123870676069, 'gamma': 0.7503265643684676, 'psi': 15}. Best is trial 1 with value: -0.8316326530612245.


...

[I 2024-09-09 18:55:45,613] Trial 14 finished with value: -0.851145038167939 and parameters: {'kernel_size': (5, 5), 'sigma': 6.619361654102951, 'theta': 60, 'lambda_': 2.108167691290082, 'gamma': 0.5386617985324982, 'psi': 30}. Best is trial 8 with value: -0.8696202531645569.


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step


[I 2024-09-09 19:05:10,859] Trial 15 finished with value: -0.761118170266836 and parameters: {'kernel_size': (11, 11), 'sigma': 8.522696632033627, 'theta': 15, 'lambda_': 3.624630230030969, 'gamma': 1.0943726283765947, 'psi': 45}. Best is trial 8 with value: -0.8696202531645569.


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step


[I 2024-09-09 19:14:12,362] Trial 16 finished with value: -0.832274459974587 and parameters: {'kernel_size': (3, 3), 'sigma': 9.119364401537663, 'theta': 0, 'lambda_': 5.659857106727527, 'gamma': 1.6352061703559029, 'psi': 0}. Best is trial 8 with value: -0.8696202531645569.
